In [1]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

/Users/vsaravind/dev/ChatterChum/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
import clip
import sys
sys.path.append("..")

In [7]:
from core.embed.clip import get_clip_embedding
from core.loader.image_loader import ImageLoader

In [8]:
loader = ImageLoader(images="../localstore/b02b6e83-21bb-4199-a405-f43aada87b75", auto_load=True)

In [26]:
embeddings = {}
for key, img in loader.iter():
    embeddings[key] = get_clip_embedding(img.image)

In [18]:
from db.qdrant_connect import QdrantConnection, QdrantCollections
from db.sql_connect import SqlConnection
from handlers.photo_handler import PhotoHandler
from models import Photo

In [16]:
handler = PhotoHandler(SqlConnection)

In [23]:
photos = await handler.list(filters={"owner_id": "b02b6e83-21bb-4199-a405-f43aada87b75", "datastore": "local"})
photos = [Photo(**photo) for photo in photos]
photos[0]

Photo(id=UUID('ab6da56e-dad3-4ad9-88eb-da05ef2c37c9'), uri='/Users/vsaravind/dev/ChatterChum/localstore/b02b6e83-21bb-4199-a405-f43aada87b75/ab6da56e-dad3-4ad9-88eb-da05ef2c37c9.jpeg', faces_processed=False, objects_processed=True, scene_processed=True, datastore=<DatastoreType.LOCAL: 'local'>, objects=[], scenes=[['outdoor', 1.0], ['rainforest', 0.36332589387893677], ['botanical_garden', 0.16723760962486267]], entities=['outdoor', 'rainforest', 'botanical garden'], owner_id=UUID('b02b6e83-21bb-4199-a405-f43aada87b75'))

In [24]:
len(photos)

20

In [28]:
with QdrantConnection(QdrantCollections.CLIP_EMBEDDINGS) as conn:
    for photo in photos:
        conn.upsert(str(photo.id), photo.model_dump(), embeddings[photo.id])

/Users/vsaravind/dev/ChatterChum/.venv/lib/python3.10/site-packages/pydantic/main.py:387: UserWarning: Pydantic serializer warnings:
  Expected `tuple[str, float]` but got `list` with value `['outdoor', 1.0]` - serialized value may not be as expected
  Expected `tuple[str, float]` but got `list` with value `['rainforest', 0.36332589387893677]` - serialized value may not be as expected
  Expected `tuple[str, float]` but got `list` with value `['botanical_garden', 0.16723760962486267]` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/Users/vsaravind/dev/ChatterChum/.venv/lib/python3.10/site-packages/pydantic/main.py:387: UserWarning: Pydantic serializer warnings:
  Expected `tuple[str, float]` but got `list` with value `['car', 0.9622276425361633]` - serialized value may not be as expected
  Expected `tuple[str, float]` but got `list` with value `['car', 0.9262083768844604]` - serialized value may not be as expected
  Expected `tuple[str, float]